In [1]:
include("../src/SphereIFSCalib.jl")
using TwoDimensional, Zygote,StatsBase, Plots
plotly()

Plots.PlotlyBackend()

In [2]:
# wavelengths
const λ1 = 987.72e-9# laser 1 
const λ2 = 1123.71e-9# laser 2 
const λ3 = 1309.37e-9# laser 3
const λ4 = 1545.10e-9  # laser 4  
const λlaser = [λ1,λ2,λ3,λ4]
const λ0 = mean(λlaser);# reference


In [3]:
# model of the LensletModel
bbox = BoundingBox(xmin=1, ymin=1, xmax=10, ymax=50);
a0= rand(Float64,4);
fwhm0= rand(Float64,4)*10;
laser =  LaserModel(λlaser,a0,fwhm0)
lmod = LensletModel(λ0,laser.nλ-1,bbox);



In [4]:
Cinit = [[ 6.2 1e6 1e6 -1e7]; [25 7e7 1e6 -1e7]]
UpdateDispModel(lmod.dmodel,Cinit)
limage = LensletLaserImage(lmod,laser)
heatmap(limage)

<!DOCTYPE html>
 
 
 Plots.jl

In [5]:
function  likelihood(a::Array{Float64,1},fwhm::Array{Float64,1},C::Array{Float64,2}) 
    UpdateDispModel(lmod.dmodel, C);
    UpdateLaserModel(laser,a,fwhm);
    return sum((data .- LensletLaserImage(lmod,laser)).^2)
end

data= limage ;
cost = likelihood(a0,fwhm0,Cinit)


0.0

In [6]:
@time ∇cost = gradient(likelihood,a0,fwhm0,Cinit)


 27.866778 seconds (58.78 M allocations: 3.356 GiB, 4.50% gc time, 99.92% compilation time)


([0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0])

In [7]:
@time  ∇cost = gradient(likelihood,a0,fwhm0 .+ 2.,Cinit)


  0.046481 seconds (144.30 k allocations: 8.602 MiB, 95.55% compilation time)


([9.692034751746416, 16.94669269819209, 2.5023236539759317, 0.9324220797944702], [1.3184514779713916, 1.6893949073432064, 0.44012541257947385, 0.007858363274520282], [-0.35224962785235636 -5.211177095092774e-8 -8.398767095890983e-15 -1.5295409150660926e-21; 0.3968120151818277 -6.316182277266799e-9 -8.664377323576415e-15 -3.814310906797267e-21])